## Part 1

In [1]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2024, day=8)

def test(method, input, expected):
    actual = method(input)
    if actual == expected:
        print(f'\t☑ - {method.__name__}({input}) = {expected} = {actual}')
    else:
        print(f'\t☐ - {method.__name__}({input}) = {expected} ≠ {actual}')

This is really weirdly worded. I'm thinking I'll create a map of frequency to antenna. Then for each antenna, I can compare all the other antenna points and calculate the antinode. Might be helpful to map from each position back to frequency.


In [2]:
from collections import defaultdict

def parse_input(input):
    
    frequency_to_point = defaultdict(list)
    point_to_frequency = defaultdict(list)

    for y, row in enumerate(input.split('\n')):
        for x, value in enumerate(row):
            if value != '.':
                frequency_to_point[value].append((x,y))
                point_to_frequency[(x,y)].append(value)
            


    return(frequency_to_point, point_to_frequency)

In [3]:
frequency_to_point, point_to_frequency = parse_input(puzzle.examples[0].input_data)
print(frequency_to_point)
print(point_to_frequency)

defaultdict(<class 'list'>, {'0': [(8, 1), (5, 2), (7, 3), (4, 4)], 'A': [(6, 5), (8, 8), (9, 9)]})
defaultdict(<class 'list'>, {(8, 1): ['0'], (5, 2): ['0'], (7, 3): ['0'], (4, 4): ['0'], (6, 5): ['A'], (8, 8): ['A'], (9, 9): ['A']})


I should find the distance between the two points, double it and check that coordinate. [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) gives the formula: $$ d(p,q) = \sqrt{(p_2 - q_1)^2 + (p_2 - q_1)^2} $$

Great news is that it's in the `Math` library now! Cool find! This is what I enjoy about Advent of Code is learning things like this. [Math.dist(p,q)](https://docs.python.org/3/library/math.html#math.dist)

I'm going to save this because it makes it pretty clear how to get a 3rd point on the line:

![Euclidean distance 2d](https://upload.wikimedia.org/wikipedia/commons/5/55/Euclidean_distance_2d.svg)

In [4]:
import itertools
import math

frequency_to_point, point_to_frequency = parse_input(puzzle.examples[0].input_data)

for frequency, antenna_points in frequency_to_point.items():
    
    for antenna_pairs in list(itertools.combinations(antenna_points, 2)):
        dist = math.dist(antenna_pairs[0], antenna_pairs[1])
        print(frequency, antenna_pairs)
        print(dist)


0 ((8, 1), (5, 2))
3.1622776601683795
0 ((8, 1), (7, 3))
2.23606797749979
0 ((8, 1), (4, 4))
5.0
0 ((5, 2), (7, 3))
2.23606797749979
0 ((5, 2), (4, 4))
2.23606797749979
0 ((7, 3), (4, 4))
3.1622776601683795
A ((6, 5), (8, 8))
3.605551275463989
A ((6, 5), (9, 9))
5.0
A ((8, 8), (9, 9))
1.4142135623730951


In [5]:
import itertools
import math

frequency_to_point = {'a': [(4, 3), (5, 5)]}

for frequency, antenna_points in frequency_to_point.items():
    
    for p, q in list(itertools.combinations(antenna_points, 2)):
        dist = math.dist(p, q)
        print(frequency, (p, q))
        print(dist)
 
        dx = q[0] - p[0]
        dy = q[1] - p[1]


        # o is before p and r is after q.... get it??
        o = (p[0] - dx, p[1] - dy)
        r = (q[0] + dx, q[1] + dy)

        print(o, r)



a ((4, 3), (5, 5))
2.23606797749979
(3, 1) (6, 7)


Awesome, that passed the sanity check. This is likely a valid approach.

In [6]:
import itertools
import math

def parse_input(input):
    
    frequency_to_point = defaultdict(list)
    point_to_frequency = defaultdict(list)

    bounds = (None, None)

    for y, row in enumerate(input.split('\n')):
        for x, value in enumerate(row):
            if value != '.':
                frequency_to_point[value].append((x,y))
                point_to_frequency[(x,y)].append(value)

    bounds = (x - 1, y - 1)
    return(frequency_to_point, point_to_frequency, bounds)


frequency_to_point, point_to_frequency, bounds = parse_input(puzzle.examples[0].input_data)

valid_antinodes = defaultdict(list)

for frequency, antenna_points in frequency_to_point.items():
    
    for p, q in list(itertools.combinations(antenna_points, 2)):
        dist = math.dist(p, q)
        # print(frequency, (p, q))
        # print(dist)
 
        dx = q[0] - p[0]
        dy = q[1] - p[1]


        # o is before p and r is after q.... get it??
        o = (p[0] - dx, p[1] - dy)
        r = (q[0] + dx, q[1] + dy)

        for antinode in [o, r]:
            if antinode[0] < 0 or antinode[0] > bounds[0] or antinode[1] < 0 or antinode[1] > bounds[1]:
                print(f'Invalid node: {antinode}')
            else:
                valid_antinodes[antinode].append(frequency)

print(valid_antinodes.keys())
print(len(valid_antinodes.keys()))


Invalid node: (11, 0)
Invalid node: (9, -1)
Invalid node: (12, -2)
Invalid node: (10, 11)
Invalid node: (12, 13)
dict_keys([(2, 3), (6, 5), (0, 7), (3, 1), (9, 4), (6, 0), (3, 6), (10, 2), (1, 5), (4, 2), (7, 7), (10, 10)])
12


Well, the example says there should be 14, what's the missing two?

In [7]:

def build_map(antinodes, bounds):
    map = ''
    for y in range(bounds[1]):
        for x in range(bounds[0]):
            if (x,y) in antinodes:
                map += '#'
            else:
                map += '.'
        map += '\n'
    return map

print(build_map([(2, 3), (6, 5), (0, 7), (3, 1), (9, 4), (6, 0), (3, 6), (10, 2), (1, 5), (4, 2), (7, 7), (10, 10)], (10, 10)))

......#...
...#......
....#.....
..#.......
.........#
.#....#...
...#......
#......#..
..........
..........



Let's check the example that was simpler

In [8]:
print(build_map([(3, 1), (6, 7)], (10, 10)))

..........
...#......
..........
..........
..........
..........
..........
......#...
..........
..........



In [9]:
print(bounds)

(10, 10)


D'OH!!! The bounds are wrong :(


In [10]:
import itertools
import math
from collections import defaultdict

def parse_input(input):
    
    frequency_to_point = defaultdict(list)
    point_to_frequency = defaultdict(list)

    bounds = (None, None)

    for y, row in enumerate(input.split('\n')):
        for x, value in enumerate(row):
            if value != '.':
                frequency_to_point[value].append((x,y))
                point_to_frequency[(x,y)].append(value)

    bounds = (x + 1, y + 1)
    return(frequency_to_point, point_to_frequency, bounds)


def calculate_antinode_count(input):

    frequency_to_point, point_to_frequency, bounds = parse_input(input)

    valid_antinodes = defaultdict(list)

    for frequency, antenna_points in frequency_to_point.items():
        
        for p, q in list(itertools.combinations(antenna_points, 2)):
            dist = math.dist(p, q)
            # print(frequency, (p, q))
            # print(dist)
    
            dx = q[0] - p[0]
            dy = q[1] - p[1]


            # o is before p and r is after q.... get it??
            o = (p[0] - dx, p[1] - dy)
            r = (q[0] + dx, q[1] + dy)

            for antinode in [o, r]:
                if antinode[0] < 0 or antinode[0] > bounds[0] or antinode[1] < 0 or antinode[1] > bounds[1]:
                    # print(f'Invalid node: {antinode}')
                    pass
                else:
                    valid_antinodes[antinode].append(frequency)

    return len(valid_antinodes.keys())

In [11]:
calculate_antinode_count(puzzle.examples[0].input_data)

14

In [12]:
calculate_antinode_count(puzzle.input_data)

296

296 is too high 

In [13]:
def build_map(antinodes, frequencies, bounds):

    map = ''
    for y in range(bounds[1]):
        for x in range(bounds[0]):
            if (x,y) in antinodes:
                map += '#'
            elif (x,y) in frequencies:
                map += frequencies[(x,y)]
            else:
                map += '.'
        map += '\n'
    return map

def calculate_antinode_count(input):

    frequency_to_point, point_to_frequency, bounds = parse_input(input)

    valid_antinodes = defaultdict(list)

    for frequency, antenna_points in frequency_to_point.items():
        
        for p, q in list(itertools.combinations(antenna_points, 2)):
            dist = math.dist(p, q)
            # print(frequency, (p, q))
            # print(dist)
    
            dx = q[0] - p[0]
            dy = q[1] - p[1]


            # o is before p and r is after q.... get it??
            o = (p[0] - dx, p[1] - dy)
            r = (q[0] + dx, q[1] + dy)

            for antinode in [o, r]:
                if antinode[0] < 0 or antinode[0] > bounds[0] or antinode[1] < 0 or antinode[1] > bounds[1]:
                    print(f'Invalid node: {antinode}')
                    pass
                else:
                    valid_antinodes[antinode].append(frequency)

            print(build_map([o,r], {p: frequency, q: frequency}, bounds))
    
    print(build_map(valid_antinodes.keys(), {}, bounds))
    return len(valid_antinodes.keys())

In [14]:
calculate_antinode_count(puzzle.examples[0].input_data)

...........#
........0...
.....0......
..#.........
............
............
............
............
............
............
............
............

Invalid node: (9, -1)
............
........0...
............
.......0....
............
......#.....
............
............
............
............
............
............

Invalid node: (12, -2)
............
........0...
............
............
....0.......
............
............
#...........
............
............
............
............

............
...#........
.....0......
.......0....
.........#..
............
............
............
............
............
............
............

......#.....
............
.....0......
............
....0.......
............
...#........
............
............
............
............
............

............
............
..........#.
.......0....
....0.......
.#..........
............
............
............
............
............
............

............


14

In [15]:
calculate_antinode_count(puzzle.input_data)

Invalid node: (-9, -7)
...d..............................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
...............d..................................
..................................................
..................................................
..................................................
..................................................
..................................................
..................................................
...........................#......................
..................................................
..................................................
..................................................
..................................................
........

296

When I copy that map into a text editor window and search, it says there are fewer '#' characters found. Is something weird going on with the keys?

In [16]:
def calculate_antinode_count(input):

    frequency_to_point, point_to_frequency, bounds = parse_input(input)


    valid_antinodes = defaultdict(list)

    for frequency, antenna_points in frequency_to_point.items():
        
        for p, q in list(itertools.combinations(antenna_points, 2)):
            dist = math.dist(p, q)
            # print(frequency, (p, q))
            # print(dist)
    
            dx = q[0] - p[0]
            dy = q[1] - p[1]


            # o is before p and r is after q.... get it??
            o = (p[0] - dx, p[1] - dy)
            r = (q[0] + dx, q[1] + dy)

            for antinode in [o, r]:
                if antinode[0] < 0 or antinode[0] > bounds[0] or antinode[1] < 0 or antinode[1] > bounds[1]:
                    # print(f'Invalid node: {antinode}')
                    pass
                else:
                    valid_antinodes[antinode].append(frequency)

            # print(build_map([o,r], {p: frequency, q: frequency}, bounds))0
    
    print(len(valid_antinodes.keys()))
    print(len(set(valid_antinodes.keys())))
    print(sorted(valid_antinodes.keys()))
    return len(valid_antinodes.keys())

calculate_antinode_count(puzzle.input_data)



296
296
[(0, 21), (0, 24), (0, 35), (0, 42), (1, 4), (1, 11), (1, 17), (1, 18), (1, 22), (1, 25), (1, 33), (1, 44), (1, 49), (1, 50), (2, 1), (2, 31), (2, 33), (2, 44), (2, 46), (3, 9), (3, 11), (3, 24), (3, 26), (3, 33), (3, 45), (4, 4), (4, 9), (4, 25), (4, 28), (5, 6), (5, 13), (5, 17), (5, 34), (5, 47), (5, 48), (5, 49), (6, 9), (6, 15), (6, 16), (6, 39), (6, 46), (7, 13), (7, 16), (7, 18), (7, 24), (7, 33), (7, 36), (8, 2), (8, 8), (8, 15), (8, 16), (8, 39), (9, 6), (9, 18), (9, 23), (9, 27), (9, 33), (10, 9), (10, 10), (10, 18), (10, 23), (10, 28), (10, 46), (11, 5), (11, 17), (11, 22), (11, 36), (11, 47), (12, 2), (12, 8), (12, 10), (12, 24), (12, 31), (12, 40), (12, 44), (12, 46), (13, 15), (13, 17), (13, 18), (13, 32), (13, 35), (13, 41), (13, 48), (14, 7), (14, 10), (14, 14), (14, 15), (14, 20), (14, 27), (14, 29), (14, 50), (15, 7), (15, 14), (15, 20), (15, 30), (16, 3), (16, 7), (16, 9), (16, 12), (16, 21), (16, 23), (16, 32), (16, 42), (17, 17), (17, 20), (17, 28), (17, 35

296

In [17]:
def build_map(antinodes, frequencies, bounds):
    found_antinodes = []
    map = ''
    for y in range(bounds[1]):
        for x in range(bounds[0]):
            if (x,y) in antinodes:
                found_antinodes.append((x,y))
                map += '#'
            elif (x,y) in frequencies:
                map += frequencies[(x,y)]
            else:
                map += '.'
        map += '\n'

    print(f'NOT FOUND!: {[x for x in antinodes if x not in found_antinodes]}')
    return map

valid_antinode_keys = [(0, 21), (0, 24), (0, 35), (0, 42), (1, 4), (1, 11), (1, 17), (1, 18), (1, 22), (1, 25), (1, 33), (1, 44), (1, 49), (1, 50), (2, 1), (2, 31), (2, 33), (2, 44), (2, 46), (3, 9), (3, 11), (3, 24), (3, 26), (3, 33), (3, 45), (4, 4), (4, 9), (4, 25), (4, 28), (5, 6), (5, 13), (5, 17), (5, 34), (5, 47), (5, 48), (5, 49), (6, 9), (6, 15), (6, 16), (6, 39), (6, 46), (7, 13), (7, 16), (7, 18), (7, 24), (7, 33), (7, 36), (8, 2), (8, 8), (8, 15), (8, 16), (8, 39), (9, 6), (9, 18), (9, 23), (9, 27), (9, 33), (10, 9), (10, 10), (10, 18), (10, 23), (10, 28), (10, 46), (11, 5), (11, 17), (11, 22), (11, 36), (11, 47), (12, 2), (12, 8), (12, 10), (12, 24), (12, 31), (12, 40), (12, 44), (12, 46), (13, 15), (13, 17), (13, 18), (13, 32), (13, 35), (13, 41), (13, 48), (14, 7), (14, 10), (14, 14), (14, 15), (14, 20), (14, 27), (14, 29), (14, 50), (15, 7), (15, 14), (15, 20), (15, 30), (16, 3), (16, 7), (16, 9), (16, 12), (16, 21), (16, 23), (16, 32), (16, 42), (17, 17), (17, 20), (17, 28), (17, 35), (17, 44), (17, 48), (18, 5), (18, 17), (18, 24), (18, 27), (18, 31), (18, 37), (19, 2), (19, 10), (19, 11), (19, 17), (19, 18), (19, 19), (19, 20), (19, 29), (19, 35), (19, 39), (19, 46), (20, 3), (20, 4), (20, 12), (20, 25), (20, 27), (20, 29), (20, 41), (21, 1), (21, 14), (21, 18), (21, 21), (21, 22), (22, 16), (22, 18), (22, 22), (22, 31), (22, 50), (23, 1), (23, 13), (23, 22), (23, 23), (23, 26), (23, 31), (23, 41), (23, 42), (24, 11), (24, 20), (24, 23), (24, 38), (24, 42), (24, 47), (24, 48), (25, 3), (25, 17), (25, 23), (25, 27), (25, 32), (25, 36), (25, 41), (26, 9), (26, 15), (26, 17), (26, 18), (26, 20), (26, 21), (26, 24), (26, 32), (26, 33), (26, 46), (26, 48), (27, 4), (27, 14), (27, 16), (27, 23), (27, 33), (27, 40), (27, 49), (28, 7), (28, 8), (28, 14), (28, 28), (28, 33), (28, 39), (28, 42), (29, 43), (30, 4), (30, 8), (30, 26), (30, 39), (30, 49), (31, 8), (31, 19), (31, 22), (31, 35), (31, 36), (31, 43), (32, 21), (32, 36), (32, 40), (32, 43), (32, 44), (33, 24), (33, 30), (33, 36), (33, 38), (34, 15), (34, 16), (34, 22), (34, 35), (34, 36), (34, 37), (34, 40), (34, 47), (34, 49), (35, 7), (35, 16), (35, 18), (35, 28), (35, 30), (35, 34), (36, 5), (36, 26), (36, 28), (36, 34), (36, 43), (37, 2), (37, 12), (37, 15), (37, 27), (37, 44), (37, 47), (38, 0), (38, 5), (38, 8), (38, 13), (38, 16), (38, 23), (38, 30), (39, 10), (39, 27), (39, 35), (39, 41), (40, 2), (40, 9), (40, 14), (40, 41), (41, 7), (41, 17), (41, 18), (42, 5), (42, 19), (42, 25), (42, 28), (42, 31), (42, 43), (43, 7), (43, 12), (43, 20), (43, 28), (43, 31), (43, 32), (44, 2), (44, 10), (44, 30), (44, 40), (45, 19), (45, 24), (45, 35), (45, 46), (46, 22), (46, 30), (46, 47), (47, 17), (47, 18), (47, 24), (47, 29), (47, 39), (47, 44), (48, 25), (48, 31), (48, 41), (48, 47), (49, 3), (49, 4), (49, 25), (49, 35), (49, 43), (49, 46), (50, 21), (50, 39)]

print(build_map(valid_antinode_keys, {}, (50, 50)))

NOT FOUND!: [(1, 50), (14, 50), (22, 50), (50, 21), (50, 39)]
......................................#...........
..#..................#.#..........................
........#...#......#.................#..#...#.....
................#...#....#.......................#
.#..#...............#......#..#..................#
...........#......#.................#.#...#.......
.....#...#........................................
..............###...........#......#.....#.#......
........#...#...............#.##......#...........
...##.#...#.....#.........#.............#.........
..........#.#.#....#...................#....#.....
.#.#...............#....#.........................
................#...#................#.....#......
.....#.#...............#..............#...........
..............##.....#.....##...........#.........
......#.#....##...........#.......#..#............
......###.............#....#......##..#...........
.#...#.....#.#...###.....##..............#.....#..
.#.....#.##..#.....#

Welp... Bounds checking got me. Easy enough fix.

In [18]:
def calculate_antinode_count(input):

    frequency_to_point, point_to_frequency, bounds = parse_input(input)


    valid_antinodes = defaultdict(list)

    for frequency, antenna_points in frequency_to_point.items():
        
        for p, q in list(itertools.combinations(antenna_points, 2)):
            dist = math.dist(p, q)
            # print(frequency, (p, q))
            # print(dist)
    
            dx = q[0] - p[0]
            dy = q[1] - p[1]


            # o is before p and r is after q.... get it??
            o = (p[0] - dx, p[1] - dy)
            r = (q[0] + dx, q[1] + dy)

            for antinode in [o, r]:
                if antinode[0] < 0 or antinode[0] >= bounds[0] or antinode[1] < 0 or antinode[1] >= bounds[1]:
                    # print(f'Invalid node: {antinode}')
                    pass
                else:
                    valid_antinodes[antinode].append(frequency)

            # print(build_map([o,r], {p: frequency, q: frequency}, bounds))0

    return len(valid_antinodes.keys())

calculate_antinode_count(puzzle.input_data)

291

## Part 2

Let me just put this here... [Collinearity](https://en.wikipedia.org/wiki/Collinearity)

These articles were great: 
* amWhy (https://math.stackexchange.com/users/9003/amwhy), Methods for showing three points in $\mathbb{R}^2$ are colinear (or not), URL (version: 2012-11-19): https://math.stackexchange.com/q/38358
* Librecoin (https://math.stackexchange.com/users/53846/librecoin), If I have three points, is there an easy way to tell if they are collinear?, URL (version: 2017-04-13): https://math.stackexchange.com/q/406009

I understand the slope method and it's interesting that it doesn't work for a straight line on the x axis. 

In reading through the page on wikipedia on [determinats](https://en.wikipedia.org/wiki/Determinant), I recall them from college math (and Leibniz rang the bell), but they're not very comfortable. I'll have to keep reading to recall enough to understand the area formula being given for the triangle.

I could use the geometric [Heron's formula](https://en.wikipedia.org/wiki/Heron%27s_formula) 

![Triangle with notations 2 without points](https://upload.wikimedia.org/wikipedia/commons/e/eb/Triangle_with_notations_2_without_points.svg)

$$ A = \sqrt{s(s - a)(s - b)(s - c)} $$ 

$$ s = \frac{a + b + c}{2} $$

I know the points. Will calculating the distance prove to be an issue? Let's find out!


In [19]:
import itertools
import math
from collections import defaultdict

def parse_input(input):
    
    frequency_to_point = defaultdict(list)

    bounds = (None, None)

    for y, row in enumerate(input.split('\n')):
        for x, value in enumerate(row):
            if value != '.':
                frequency_to_point[value].append((x,y))

    bounds = (x + 1, y + 1)
    return(frequency_to_point, bounds)


def calculate_antinode_count(input):

    frequency_to_point, bounds = parse_input(input)

    valid_antinodes = defaultdict(list)

    nodes = [(x, y) for x in range(bounds[0]) for y in range(bounds[1])]

    print(nodes)

    for frequency, antenna_points in frequency_to_point.items():
        
        for p, q in list(itertools.combinations(antenna_points, 2)):
            c = math.dist(p, q)
    
            for r in nodes:
                b = math.dist(p, r)
                a = math.dist(q, r)

                s = .5 * (a + b + c)

                radicand = s*(s-a)*(s-b)*(s-c)

                if (radicand) < 0:
                    if (r == (0,0)):
                        print('sqrt was undefined: {}')
                    continue
                A = math.sqrt(radicand)
                if (r == (0,0)):
                    print(A)
                    print(p, q, r)
                if A == 0:
                    valid_antinodes[r] = frequency

    print(build_map(valid_antinodes.keys(), {}, bounds))
    return len(valid_antinodes.keys())



calculate_antinode_count(puzzle.examples[0].input_data)

[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (6, 11), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7), (7, 8), (7, 9), (7, 10), (7, 11), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (8, 9), (8, 10), (8, 11), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9), (9, 10), (9, 11), (10, 0), (10, 1), (

31

This is pretty cool! I'm getting a floating point precision error. 


`(8, 8)` `(9, 9)` `(0, 0)` are certainly colinear, but the area is coming out to `6.014559854891645e-07`.

In [20]:
import itertools
import math
from collections import defaultdict
from decimal import Decimal, getcontext

def parse_input(input):
    
    frequency_to_point = defaultdict(list)

    bounds = (None, None)

    for y, row in enumerate(input.split('\n')):
        for x, value in enumerate(row):
            if value != '.':
                frequency_to_point[value].append((Decimal(x),Decimal(y)))

    bounds = (x + 1, y + 1)
    return(frequency_to_point, bounds)


def calculate_antinode_count(input):

    frequency_to_point, bounds = parse_input(input)

    valid_antinodes = defaultdict(list)

    nodes = [(Decimal(x), Decimal(y)) for x in range(bounds[0]) for y in range(bounds[1])]

    print(nodes)
    getcontext().prec = 50
    for frequency, antenna_points in frequency_to_point.items():
        
        for p, q in list(itertools.combinations(antenna_points, 2)):
            c = Decimal(math.dist(p, q))
    
            for r in nodes:
                b = Decimal(math.dist(p, r))
                a = Decimal(math.dist(q, r))

                s = Decimal(.5) * (a + b + c)

                radicand = s*(s-a)*(s-b)*(s-c)

                if (radicand) < 0:
                    if (r == (0,0)):
                        print('sqrt was undefined: {}')
                    continue
                A = math.sqrt(radicand)
                if (r == (0,0)):
                    print(A)
                    print(p, q, r)
                if A == 0:
                    valid_antinodes[r] = frequency

    print(build_map(valid_antinodes.keys(), {}, bounds))
    return len(valid_antinodes.keys())



calculate_antinode_count(puzzle.examples[0].input_data)

[(Decimal('0'), Decimal('0')), (Decimal('0'), Decimal('1')), (Decimal('0'), Decimal('2')), (Decimal('0'), Decimal('3')), (Decimal('0'), Decimal('4')), (Decimal('0'), Decimal('5')), (Decimal('0'), Decimal('6')), (Decimal('0'), Decimal('7')), (Decimal('0'), Decimal('8')), (Decimal('0'), Decimal('9')), (Decimal('0'), Decimal('10')), (Decimal('0'), Decimal('11')), (Decimal('1'), Decimal('0')), (Decimal('1'), Decimal('1')), (Decimal('1'), Decimal('2')), (Decimal('1'), Decimal('3')), (Decimal('1'), Decimal('4')), (Decimal('1'), Decimal('5')), (Decimal('1'), Decimal('6')), (Decimal('1'), Decimal('7')), (Decimal('1'), Decimal('8')), (Decimal('1'), Decimal('9')), (Decimal('1'), Decimal('10')), (Decimal('1'), Decimal('11')), (Decimal('2'), Decimal('0')), (Decimal('2'), Decimal('1')), (Decimal('2'), Decimal('2')), (Decimal('2'), Decimal('3')), (Decimal('2'), Decimal('4')), (Decimal('2'), Decimal('5')), (Decimal('2'), Decimal('6')), (Decimal('2'), Decimal('7')), (Decimal('2'), Decimal('8')), (Deci

18

Got the error smaller, but not vanished.

`3.3622411725697453e-07`

In [21]:
import itertools
import math
from collections import defaultdict

def parse_input(input):
    
    frequency_to_point = defaultdict(list)

    bounds = (None, None)

    for y, row in enumerate(input.split('\n')):
        for x, value in enumerate(row):
            if value != '.':
                frequency_to_point[value].append((x,y))

    bounds = (x + 1, y + 1)
    return(frequency_to_point, bounds)


def calculate_antinode_count(input):

    frequency_to_point, bounds = parse_input(input)

    valid_antinodes = defaultdict(list)

    nodes = [(x, y) for x in range(bounds[0]) for y in range(bounds[1])]

    # print(nodes)

    for frequency, antenna_points in frequency_to_point.items():
        
        for p, q in list(itertools.combinations(antenna_points, 2)):
            c = math.dist(p, q)
    
            for r in nodes:
                b = math.dist(p, r)
                a = math.dist(q, r)

                s = 0.5 * (a + b + c)

                radicand = s*(s-a)*(s-b)*(s-c)

                if (radicand) < 0:
                    # if (r == (0,0)):
                    #     print('sqrt was undefined: {}')
                    continue
                A = math.sqrt(radicand)
                # if (r == (0,0)):
                #     print(A)
                #     print(p, q, r)
                if A == 0 or A < 1e-5:
                    valid_antinodes[r] = frequency

    # print(build_map(valid_antinodes.keys(), {}, bounds))
    return len(valid_antinodes.keys())

In [22]:
calculate_antinode_count(puzzle.examples[0].input_data)

34

In [23]:

calculate_antinode_count(puzzle.input_data)

960

`960` is too low.

I will have to switch to the determinant area varient :(

In [24]:
import itertools
import math
from collections import defaultdict

def parse_input(input):
    
    frequency_to_point = defaultdict(list)

    bounds = (None, None)

    for y, row in enumerate(input.split('\n')):
        for x, value in enumerate(row):
            if value != '.':
                frequency_to_point[value].append((x,y))

    bounds = (x + 1, y + 1)
    return(frequency_to_point, bounds)


def calculate_antinode_count(input):

    frequency_to_point, bounds = parse_input(input)

    valid_antinodes = defaultdict(list)

    nodes = [(x, y) for x in range(bounds[0]) for y in range(bounds[1])]

    # print(nodes)

    for frequency, antenna_points in frequency_to_point.items():
        
        for p, q in list(itertools.combinations(antenna_points, 2)):
            c = math.dist(p, q)
    
            for r in nodes:
                a, b = p
                m, n = q
                x, y = r
                # area = p[0] * (q[0] - r[1]) + q[1] * (r[1] - p[1]) + r[0] * (p[1] - q[0])
                area = a*(n-y) + m*(y-b) + x*(b-n)
                if area == 0:
                    valid_antinodes[r] = frequency

    # print(build_map(valid_antinodes.keys(), {}, bounds))
    return len(valid_antinodes.keys())

In [25]:
calculate_antinode_count(puzzle.examples[0].input_data)

34

In [26]:

calculate_antinode_count(puzzle.input_data)

1015